# 🎯 ConvNeXtPose Testing en Kaggle - Modelos L y M

Este notebook evalúa los modelos ConvNeXtPose L y M en Human3.6M Protocol 2.

**Datasets requeridos:**
- Human3.6M Dataset (con S9_ACT2_16, S11_ACT2_16, annotations)
- ConvNeXtPose Pre-trained Models (checkpoints .tar)

**GPU recomendada:** T4 x2 o P100

## 📦 PASO 1: Setup Inicial

In [1]:
!pwd

/kaggle/working


In [2]:
#!rm -r ConvNeXtPose 

In [3]:
# Clonar repositorio
!git clone https://github.com/EstebanCabreraArbizu/ConvNeXtPose.git
%cd ConvNeXtPose
!git fetch origin

# Verificar versiones
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memoria GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

Cloning into 'ConvNeXtPose'...
remote: Enumerating objects: 1166, done.
remote: Counting objects: 100% (288/288), done.
remote: Compressing objects: 100% (225/225), done.
remote: Total 1166 (delta 103), reused 222 (delta 62), pack-reused 878 (from 1)
Receiving objects: 100% (1166/1166), 318.73 MiB | 39.40 MiB/s, done.
Resolving deltas: 100% (341/341), done.
Updating files: 100% (207/207), done.
/kaggle/working/ConvNeXtPose
PyTorch: 2.5.1+cu121
CUDA disponible: True
GPU: Tesla T4
Memoria GPU: 15.83 GB


## 🗂️ PASO 2: Configurar Dataset Human3.6M

Usamos enlaces simbólicos para evitar copiar ~30GB de datos.

In [4]:
# IMPORTANTE: Ajusta este path al nombre real de tu dataset en Kaggle
KAGGLE_DATASET_PATH = '/kaggle/input/human3-6m-for-convnextpose-and-3dmpee-pose-net'  # ← CAMBIAR SEGÚN TU DATASET

# Verificar que el dataset existe
import os
if not os.path.exists(KAGGLE_DATASET_PATH):
    print(f"❌ Dataset no encontrado en {KAGGLE_DATASET_PATH}")
    print("\n📂 Datasets disponibles:")
    !ls /kaggle/input/
    raise FileNotFoundError("Verifica el nombre del dataset y actualiza KAGGLE_DATASET_PATH")
else:
    print(f"✓ Dataset encontrado en {KAGGLE_DATASET_PATH}")
    print("\n📂 Contenido:")
    !ls {KAGGLE_DATASET_PATH}

✓ Dataset encontrado en /kaggle/input/human3-6m-for-convnextpose-and-3dmpee-pose-net

📂 Contenido:
'annotations (1)'   S11_ACT2_16   S9_ACT2_16


### (Opcional) Diagnosticar Estructura del Dataset

Si tienes dudas sobre la estructura de tu dataset, ejecuta esta celda primero para ver cómo están organizadas las carpetas.

In [5]:
# Diagnosticar estructura del dataset (útil para identificar carpetas anidadas)
# !python diagnose_kaggle_dataset.py {KAGGLE_DATASET_PATH}

In [6]:
# Ejecutar script de configuración
# IMPORTANTE: Esto enlaza el dataset DENTRO de data/Human36M/, NO reemplaza data/
# El script detecta automáticamente carpetas anidadas (ej: annotations (1)/annotations/)
!python setup_kaggle_dataset.py --kaggle-input {KAGGLE_DATASET_PATH} --project-root /kaggle/working/ConvNeXtPose

print("\n✅ Dataset enlazado en data/Human36M/")
print("✅ Módulos Python originales intactos en data/")
print("✅ Carpetas anidadas detectadas automáticamente")


  Configuración de Dataset Human3.6M para ConvNeXtPose
📂 Dataset Kaggle:     /kaggle/input/human3-6m-for-convnextpose-and-3dmpee-pose-net
📂 Proyecto ConvNeXt:  /kaggle/working/ConvNeXtPose

✓ Directorio del proyecto encontrado: /kaggle/working/ConvNeXtPose/data/Human36M
✓ Manteniendo módulos Python originales en /kaggle/working/ConvNeXtPose/data

📁 [1/3] Configurando annotations...
  ✓ Encontrado: annotations (1)/annotations
    (21 archivos JSON detectados)
  ✓ Creado: annotations -> /kaggle/input/human3-6m-for-convnextpose-and-3dmpee-pose-net/annotations (1)/annotations

👥 [2/3] Configurando sujetos S9 y S11...
  ✓ Creado: S9 -> /kaggle/input/human3-6m-for-convnextpose-and-3dmpee-pose-net/S9_ACT2_16
  ✓ Creado: S11 -> /kaggle/input/human3-6m-for-convnextpose-and-3dmpee-pose-net/S11_ACT2_16

📦 [3/3] Configurando bbox_root...
  ⚠️  No se encontró bbox_root (opcional)

  ✅ Configuración Completada

📂 Estructura creada en: /kaggle/working/ConvNeXtPose/data/Human36M

Contenido:
  📁 __pyc

In [7]:
from pathlib import Path

images_dir = Path('/kaggle/working/ConvNeXtPose/data/Human36M/images')

for subject in ['S9', 'S11']:
    subj_dir = images_dir / subject
    if not subj_dir.exists():
        continue
    for seq_dir in subj_dir.iterdir():
        target = images_dir / seq_dir.name
        if target.exists():
            continue      # ya estaba creado
        target.symlink_to(seq_dir, target_is_directory=True)

In [8]:
!find /kaggle/working/ConvNeXtPose/data/Human36M/images -maxdepth 1 -type l | head

/kaggle/working/ConvNeXtPose/data/Human36M/images/s_09_act_05_subact_01_ca_04
/kaggle/working/ConvNeXtPose/data/Human36M/images/s_09_act_13_subact_02_ca_02
/kaggle/working/ConvNeXtPose/data/Human36M/images/s_09_act_15_subact_02_ca_03
/kaggle/working/ConvNeXtPose/data/Human36M/images/s_09_act_10_subact_02_ca_02
/kaggle/working/ConvNeXtPose/data/Human36M/images/s_09_act_02_subact_01_ca_04
/kaggle/working/ConvNeXtPose/data/Human36M/images/s_11_act_16_subact_02_ca_01
/kaggle/working/ConvNeXtPose/data/Human36M/images/s_11_act_08_subact_02_ca_04
/kaggle/working/ConvNeXtPose/data/Human36M/images/s_11_act_06_subact_01_ca_01
/kaggle/working/ConvNeXtPose/data/Human36M/images/s_11_act_12_subact_01_ca_02
/kaggle/working/ConvNeXtPose/data/Human36M/images/s_11_act_03_subact_02_ca_03
find: ‘standard output’: Broken pipe
find: write error


In [9]:
# Verificar que la estructura es correcta
!python setup_kaggle_dataset.py --verify /kaggle/working/ConvNeXtPose


  🔍 Verificación de Estructura
  ✓ data/dataset.py
  ✓ data/Human36M/Human36M.py
  ✓ data/Human36M/annotations
  ✓ data/Human36M/images
  ✓ data/Human36M/images/S9
  ✓ data/Human36M/images/S11
  ✓ data/Human36M/bbox_root (optional)

  ✅ Estructura verificada correctamente
  ✅ Módulos Python intactos en data/
  ✅ Dataset enlazado en data/Human36M/


In [10]:
# Verificar que la estructura es correcta
!python verify_kaggle_structure.py


  🔍 VERIFICACIÓN RÁPIDA - Estructura del Proyecto

📄 Módulos Python originales:
  ✓ data/dataset.py
  ✓ data/multiple_datasets.py
  ✓ data/Human36M/Human36M.py
  ✓ common/base.py
  ✓ main/config.py

📂 Dataset de Kaggle (enlaces):
  ✓ data/Human36M/images/S9
  ✓ data/Human36M/images/S11
  ✓ data/Human36M/annotations
  ✓ data/Human36M/bbox_root

✅ ESTRUCTURA CORRECTA - Listo para testing

🚀 Siguiente paso:
   %cd /kaggle/working/ConvNeXtPose/main
   !python test.py --gpu 0 --epochs 83 --variant L



## 🎯 PASO 3: Preparar Checkpoints

Extraer los modelos pre-entrenados.

In [11]:
!ls

ARCHITECTURE_ADAPTATION_COMPLETE.md  KAGGLE_TESTING_GUIDE.md
assets				     kaggle_testing_notebook.ipynb
AUTHOR_CONTACT_GUIDE.md		     LICENSE
CHECKLIST_TESTING.md		     list_google_drive_contents.py
CHECKPOINT_EXTRACTION_FIX.md	     log2.txt
CHECKPOINT_INVESTIGATION_REPORT.md   log.txt
CHECKPOINT_MISLABELING_ISSUE.md      main
common				     NESTED_FOLDERS_SOLUTION.md
CORRECCION_CONFIG_S.md		     output
data				     PASOS_TESTING.md
demo				     PLAN_ACCION_INMEDIATO.md
diagnose_kaggle_dataset.py	     quick_start.sh
EMAIL_TEMPLATE_AUTHORS.md	     README.md
ESTADO_PROYECTO.md		     README_TESTING.md
EXPLICACION_DIMS_INCORRECTOS.md      requirements.txt
exports				     RESUMEN_EJECUTIVO.md
GITHUB_ISSUE_TEMPLATE.md	     RESUMEN_RETESTING.md
GUIA_TESTING_MODELOS_L_M.md	     setup_kaggle_dataset.py
identify_model_variant.py	     tool
KAGGLE_DATASET_FIX.md		     UBUNTU_QUICKSTART.md
KAGGLE_EXECUTION_GUIDE.md	     ubuntu_quickstart.sh
KAGGLE_QUICK_SOLUTION.md	     verify_kaggle_structure.py

In [12]:
!pip install gdown

In [13]:
import gdown

folder_id = "12H7zkLvmJtrkCmAUAPkQ6788WAnO60gI"
output = "models_tar"

gdown.download_folder(id=folder_id, output=output, quiet=False, use_cookies=False)

Retrieving folder contents


Processing file 1eIaMqTYG-30CuPULs9LzSfeouAzYYHQW ConvNeXtPose_L.tar
Processing file 1X_H-6S4xrQjW9GhJ3yWB-AXqUHddvkOI ConvNeXtPose_M.tar
Processing file 1OriQPQ3uRY8MWPHP9KnwPaKawzrontqH ConvNeXtPose_S.tar
Processing file 165D0rU2GImmRe7u7DNe6eKJG8voBIcGh ConvNeXtPose_XS.tar


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1eIaMqTYG-30CuPULs9LzSfeouAzYYHQW
To: /kaggle/working/ConvNeXtPose/models_tar/ConvNeXtPose_L.tar
100%|██████████| 101M/101M [00:02<00:00, 49.1MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1X_H-6S4xrQjW9GhJ3yWB-AXqUHddvkOI
To: /kaggle/working/ConvNeXtPose/models_tar/ConvNeXtPose_M.tar
100%|██████████| 91.3M/91.3M [00:01<00:00, 68.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1OriQPQ3uRY8MWPHP9KnwPaKawzrontqH
To: /kaggle/working/ConvNeXtPose/models_tar/ConvNeXtPose_S.tar
100%|██████████| 89.6M/89.6M [00:00<00:00, 110MB/s] 
Downloading...
From: https://drive.google.com/uc?id=165D0rU2GImmRe7u7DNe6eKJG8voBIcGh
To: /kaggle/working/ConvNeXtPose/models_tar/ConvNeXtPose_XS.tar
100%|██████████| 42.5M/42.5M [00:00<00:00, 172MB/s]
Download completed


['models_tar/ConvNeXtPose_L.tar',
 'models_tar/ConvNeXtPose_M.tar',
 'models_tar/ConvNeXtPose_S.tar',
 'models_tar/ConvNeXtPose_XS.tar']

In [14]:
!ls

ARCHITECTURE_ADAPTATION_COMPLETE.md  kaggle_testing_notebook.ipynb
assets				     LICENSE
AUTHOR_CONTACT_GUIDE.md		     list_google_drive_contents.py
CHECKLIST_TESTING.md		     log2.txt
CHECKPOINT_EXTRACTION_FIX.md	     log.txt
CHECKPOINT_INVESTIGATION_REPORT.md   main
CHECKPOINT_MISLABELING_ISSUE.md      models_tar
common				     NESTED_FOLDERS_SOLUTION.md
CORRECCION_CONFIG_S.md		     output
data				     PASOS_TESTING.md
demo				     PLAN_ACCION_INMEDIATO.md
diagnose_kaggle_dataset.py	     quick_start.sh
EMAIL_TEMPLATE_AUTHORS.md	     README.md
ESTADO_PROYECTO.md		     README_TESTING.md
EXPLICACION_DIMS_INCORRECTOS.md      requirements.txt
exports				     RESUMEN_EJECUTIVO.md
GITHUB_ISSUE_TEMPLATE.md	     RESUMEN_RETESTING.md
GUIA_TESTING_MODELOS_L_M.md	     setup_kaggle_dataset.py
identify_model_variant.py	     tool
KAGGLE_DATASET_FIX.md		     UBUNTU_QUICKSTART.md
KAGGLE_EXECUTION_GUIDE.md	     ubuntu_quickstart.sh
KAGGLE_QUICK_SOLUTION.md	     verify_kaggle_structure.py
KAGGLE_QUICK

In [15]:
!ls src

ls: cannot access 'src': No such file or directory


In [16]:
!pwd

/kaggle/working/ConvNeXtPose


In [17]:
!ls -lh /kaggle/working/ConvNeXtPose/models_tar

total 310M
-rw-r--r-- 1 root root 97M Oct  4  2023 ConvNeXtPose_L.tar
-rw-r--r-- 1 root root 88M Oct  4  2023 ConvNeXtPose_M.tar
-rw-r--r-- 1 root root 86M Oct  4  2023 ConvNeXtPose_S.tar
-rw-r--r-- 1 root root 41M Oct  4  2023 ConvNeXtPose_XS.tar


In [18]:
import os
# IMPORTANTE: Ajusta según el nombre de tu dataset de modelos
MODELS_DATASET_PATH = '/kaggle/working/ConvNeXtPose/models_tar'
# Verificar modelos disponibles
if os.path.exists(MODELS_DATASET_PATH):
    print("📦 Modelos disponibles:")
    !ls -lh {MODELS_DATASET_PATH}
else:
    print(f"❌ Dataset de modelos no encontrado: {MODELS_DATASET_PATH}")
    print("\n📂 Datasets disponibles:")
    !ls /kaggle/input/

📦 Modelos disponibles:
total 310M
-rw-r--r-- 1 root root 97M Oct  4  2023 ConvNeXtPose_L.tar
-rw-r--r-- 1 root root 88M Oct  4  2023 ConvNeXtPose_M.tar
-rw-r--r-- 1 root root 86M Oct  4  2023 ConvNeXtPose_S.tar
-rw-r--r-- 1 root root 41M Oct  4  2023 ConvNeXtPose_XS.tar


In [19]:
import os
import zipfile
import shutil
import tarfile
import torch

# Crear directorio para modelos
os.makedirs('output/model_dump', exist_ok=True)

# Función para extraer y convertir checkpoints
def extract_checkpoint(tar_path, model_name, expected_epoch=None):
    """Extrae checkpoint desde .tar/.zip y lo convierte en archivo .pth válido
    
    Los archivos .tar de ConvNeXtPose son ZIP con estructura de directorio en formato legacy.
    Usamos una conversión simple: empaquetar el directorio como TAR que PyTorch puede leer.
    """
    if not os.path.exists(tar_path):
        print(f"⚠️  Modelo {model_name} no encontrado en {tar_path}")
        return None
    
    print(f"📦 Extrayendo modelo {model_name} desde {tar_path}...")
    
    # Verificar tamaño del archivo
    file_size = os.path.getsize(tar_path)
    print(f"   📏 Tamaño del archivo: {file_size / (1024*1024):.2f} MB")
    
    # Leer primeros bytes para diagnosticar el formato
    with open(tar_path, 'rb') as f:
        header = f.read(512)
        print(f"   🔍 Primeros bytes (hex): {header[:50].hex()}")
    
    # Si el archivo es muy pequeño o empieza con HTML, es un error de descarga
    if file_size < 1000 or header.startswith(b'<!DOCTYPE') or header.startswith(b'<html'):
        print(f"   ❌ ERROR: El archivo parece ser HTML (error de descarga)")
        print(f"   💡 Solución: Revisa que el folder de Google Drive sea público")
        return None
    
    # Extraer a directorio temporal
    temp_dir = 'output/model_dump/temp_extract'
    os.makedirs(temp_dir, exist_ok=True)
    
    # Detectar formato: intentar ZIP primero, luego TAR
    extracted = False
    try:
        with zipfile.ZipFile(tar_path, 'r') as zip_ref:
            zip_ref.extractall(temp_dir)
            files = zip_ref.namelist()
            print(f"   ✓ Formato: ZIP - Extraído: {len(files)} archivos")
            extracted = True
    except zipfile.BadZipFile:
        try:
            with tarfile.open(tar_path, 'r') as tar_ref:
                tar_ref.extractall(temp_dir)
                files = tar_ref.getnames()
                print(f"   ✓ Formato: TAR - Extraído: {len(files)} archivos")
                extracted = True
        except (tarfile.ReadError, Exception) as e:
            print(f"   ❌ ERROR: No se pudo extraer el archivo")
            print(f"   💡 Formato no reconocido: {type(e).__name__}: {e}")
            shutil.rmtree(temp_dir, ignore_errors=True)
            return None
    
    if not extracted:
        shutil.rmtree(temp_dir, ignore_errors=True)
        return None
    
    # Buscar la carpeta del checkpoint (snapshot_XX.pth/ o archive/)
    found_checkpoints = []
    for item in os.listdir(temp_dir):
        item_path = os.path.join(temp_dir, item)
        if os.path.isdir(item_path):
            # Verificar que tenga data.pkl (indicador de checkpoint PyTorch)
            if os.path.exists(os.path.join(item_path, 'data.pkl')):
                found_checkpoints.append((item, item_path))
                print(f"   ✓ Checkpoint encontrado: {item}/")
    
    if not found_checkpoints:
        print(f"   ❌ No se encontró estructura de checkpoint válida")
        print(f"   📂 Contenido extraído:")
        for item in os.listdir(temp_dir)[:10]:
            print(f"      - {item}")
        shutil.rmtree(temp_dir, ignore_errors=True)
        return None
    
    # Convertir el checkpoint legacy a formato moderno
    epoch = None
    for ckpt_name, ckpt_path in found_checkpoints:
        # Determinar el epoch desde el nombre
        import re
        match = re.search(r'snapshot_(\d+)', ckpt_name)
        if match:
            epoch = match.group(1)
            final_name = f'snapshot_{epoch}.pth'
        else:
            # Si no tiene snapshot_XX, usar archive/ con epoch esperado
            if expected_epoch:
                epoch = str(expected_epoch)
                final_name = f'snapshot_{epoch}.pth'
            else:
                epoch = '0'
                final_name = f'{model_name}_checkpoint.pth'
        
        dest_path = os.path.join('output/model_dump', final_name)
        
        print(f"   🔄 Convirtiendo formato legacy → formato moderno...")
        
        try:
            # SOLUCIÓN DEFINITIVA: Cargar manualmente el formato legacy y guardar como moderno
            import pickle
            import io
            
            # Cargar data.pkl
            data_pkl_path = os.path.join(ckpt_path, 'data.pkl')
            data_dir = os.path.join(ckpt_path, 'data')
            
            # Crear un unpickler personalizado que resuelve persistent IDs
            class LegacyUnpickler(pickle.Unpickler):
                def __init__(self, file, data_dir):
                    super().__init__(file)
                    self.data_dir = data_dir
                    self.storage_cache = {}
                
                def persistent_load(self, pid):
                    # pid es una tupla: ('storage', <type>, <key>, <location>, <size>)
                    if isinstance(pid, tuple) and len(pid) >= 2 and pid[0] == 'storage':
                        typename, key = pid[1:3]
                        location = pid[3] if len(pid) > 3 else None
                        
                        # Cachear storages para evitar recargar
                        if key in self.storage_cache:
                            return self.storage_cache[key]
                        
                        # Leer el archivo de storage
                        storage_file = os.path.join(self.data_dir, str(key))
                        
                        # Leer el contenido como raw binary
                        with open(storage_file, 'rb') as f:
                            raw_data = f.read()
                            
                            # Crear UntypedStorage desde el buffer
                            untyped_storage = torch.UntypedStorage.from_buffer(raw_data, dtype=torch.uint8)
                            
                            # Mapear typename a dtype de PyTorch
                            dtype_map = {
                                'FloatStorage': torch.float32,
                                'DoubleStorage': torch.float64,
                                'HalfStorage': torch.float16,
                                'LongStorage': torch.int64,
                                'IntStorage': torch.int32,
                                'ShortStorage': torch.int16,
                                'CharStorage': torch.int8,
                                'ByteStorage': torch.uint8,
                                'BoolStorage': torch.bool,
                            }
                            
                            # Obtener dtype desde typename
                            # typename puede ser un objeto _LegacyStorageMeta, extraer el nombre
                            type_str = str(typename).split('.')[-1].replace("'>", "")
                            dtype = dtype_map.get(type_str, torch.float32)
                            
                            # Crear TypedStorage desde UntypedStorage
                            typed_storage = torch.storage.TypedStorage(
                                wrap_storage=untyped_storage,
                                dtype=dtype
                            )
                            
                            self.storage_cache[key] = typed_storage
                            return typed_storage
                    
                    raise pickle.UnpicklingError(f"unsupported persistent id: {pid}")
            
            # Cargar el checkpoint usando el unpickler personalizado
            with open(data_pkl_path, 'rb') as f:
                unpickler = LegacyUnpickler(f, data_dir)
                checkpoint = unpickler.load()
            
            # Guardar en formato moderno
            torch.save(checkpoint, dest_path)
            
            # Verificar tamaño
            size_mb = os.path.getsize(dest_path) / (1024 * 1024)
            print(f"   ✓ Archivo creado: {final_name} ({size_mb:.1f} MB)")
            
            # Verificar que se puede cargar
            test_load = torch.load(dest_path, map_location='cpu', weights_only=False)
            keys = list(test_load.keys())
            print(f"   ✓ Verificación exitosa - Keys: {keys}")
            
        except Exception as e:
            print(f"   ❌ ERROR al convertir checkpoint: {type(e).__name__}")
            print(f"      {str(e)[:200]}")
            import traceback
            traceback.print_exc()
            shutil.rmtree(temp_dir, ignore_errors=True)
            return None
    
    # Limpiar temporal
    shutil.rmtree(temp_dir, ignore_errors=True)
    print(f"   ✓ Conversión completada")
    
    return epoch

# Extraer modelo L (epoch 83 según el paper)
print("="*60)
model_l_path = f'{MODELS_DATASET_PATH}/ConvNeXtPose_L.tar'
epoch_l = extract_checkpoint(model_l_path, 'L', expected_epoch=83)

print()

# Extraer modelo M (epoch 70 según el paper)
model_m_path = f'{MODELS_DATASET_PATH}/ConvNeXtPose_M.tar'
epoch_m = extract_checkpoint(model_m_path, 'M', expected_epoch=70)
print("="*60)

# Verificar checkpoints extraídos
print("\n📂 Checkpoints disponibles:")
checkpoints = [f for f in os.listdir('output/model_dump') 
               if f.endswith('.pth') and os.path.isfile(os.path.join('output/model_dump', f))]

if checkpoints:
    for ckpt in sorted(checkpoints):
        ckpt_path = os.path.join('output/model_dump', ckpt)
        size_mb = os.path.getsize(ckpt_path) / (1024 * 1024)
        print(f"  ✓ {ckpt} ({size_mb:.1f} MB)")
        
        # Verificar contenido
        try:
            test_load = torch.load(ckpt_path, map_location='cpu', weights_only=False)
            keys = list(test_load.keys())
            print(f"    → Keys: {keys}")
            if 'network' in test_load:
                # Contar parámetros
                num_params = sum(p.numel() for p in test_load['network'].values() 
                                 if isinstance(p, torch.Tensor))
                print(f"    → ✅ Formato válido ({num_params:,} parámetros)")
            else:
                print(f"    → ⚠️  Falta key 'network'")
        except Exception as e:
            print(f"    → ❌ Error: {type(e).__name__}: {str(e)[:80]}")

# Mostrar información de epochs
if epoch_l:
    print(f"\n💡 Modelo L: Usa CHECKPOINT_EPOCH = {epoch_l}")
if epoch_m:
    print(f"💡 Modelo M: Usa CHECKPOINT_EPOCH = {epoch_m}")

if epoch_l or epoch_m:
    print("\n✅ Los checkpoints están listos para torch.load()")
    print("   Formato: PyTorch moderno (.pth)")
else:
    print("\n❌ No se pudieron extraer los checkpoints")
    print("💡 Verifica que los archivos .tar se descargaron correctamente")


📦 Extrayendo modelo L desde /kaggle/working/ConvNeXtPose/models_tar/ConvNeXtPose_L.tar...
   📏 Tamaño del archivo: 96.19 MB
   🔍 Primeros bytes (hex): 504b03040000080800000000000000000000000000000000000018000a00736e617073686f745f38332e7074682f64617461
   ✓ Formato: ZIP - Extraído: 808 archivos
   ✓ Checkpoint encontrado: snapshot_83.pth/
   🔄 Convirtiendo formato legacy → formato moderno...


<ipython-input-19-2ab6ed486f7e>:162: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  typed_storage = torch.storage.TypedStorage(


   ✓ Archivo creado: snapshot_83.pth (96.2 MB)
   ✓ Verificación exitosa - Keys: ['epoch', 'network', 'optimizer']
   ✓ Conversión completada

📦 Extrayendo modelo M desde /kaggle/working/ConvNeXtPose/models_tar/ConvNeXtPose_M.tar...
   📏 Tamaño del archivo: 87.10 MB
   🔍 Primeros bytes (hex): 504b03040000080800000000000000000000000000000000000010001200617263686976652f646174612e706b6c46420e00
   ✓ Formato: ZIP - Extraído: 808 archivos
   ✓ Checkpoint encontrado: archive/
   🔄 Convirtiendo formato legacy → formato moderno...
   ✓ Archivo creado: snapshot_70.pth (87.1 MB)
   ✓ Verificación exitosa - Keys: ['epoch', 'network', 'optimizer']
   ✓ Conversión completada

📂 Checkpoints disponibles:
  ✓ snapshot_70.pth (87.1 MB)
    → Keys: ['epoch', 'network', 'optimizer']
    → ✅ Formato válido (7,596,986 parámetros)
  ✓ snapshot_83.pth (96.2 MB)
    → Keys: ['epoch', 'network', 'optimizer']
    → ✅ Formato válido (8,391,354 parámetros)

💡 Modelo L: Usa CHECKPOINT_EPOCH = 83
💡 Modelo M: Usa CH

### ⚠️ Nota sobre Extracción de Checkpoints

Los archivos `.tar` de ConvNeXtPose son en realidad **archivos ZIP** con estructura anidada:
```
ConvNeXtPose_L.tar (archivo zip)
└── snapshot_83.pth/        ← Directorio
    ├── data.pkl
    ├── version
    └── data/               ← Carpeta con el checkpoint real
        └── 0, 1, 2...      ← Archivos binarios
```

La siguiente celda extrae y reorganiza correctamente el archivo `.pth` real.

In [20]:
!pwd

/kaggle/working/ConvNeXtPose


In [21]:
# Verificar checkpoints extraídos y detectar epoch
import glob
import re
import os
import torch

print("🔍 Verificación de Checkpoints:\n")

# Buscar ARCHIVOS .pth
checkpoint_dir = 'output/model_dump'
if os.path.exists(checkpoint_dir):
    checkpoints = [f for f in os.listdir(checkpoint_dir) 
                   if f.endswith('.pth') and os.path.isfile(os.path.join(checkpoint_dir, f))]
else:
    checkpoints = []

if checkpoints:
    print("✅ Checkpoints encontrados:\n")
    for ckpt in sorted(checkpoints):
        ckpt_path = os.path.join(checkpoint_dir, ckpt)
        size_mb = os.path.getsize(ckpt_path) / (1024 * 1024)
        
        # Extraer epoch
        match = re.search(r'snapshot_(\d+)', ckpt)
        if match:
            epoch = match.group(1)
            print(f"  ✓ {ckpt} ({size_mb:.1f} MB)")
            print(f"    → Usa CHECKPOINT_EPOCH = {epoch}")
            
            # Verificar contenido del checkpoint
            try:
                test_load = torch.load(ckpt_path, map_location='cpu', weights_only=False)
                keys = list(test_load.keys())
                print(f"    → Keys: {keys}")
                
                if 'network' in test_load:
                    # Contar parámetros del modelo
                    num_params = sum(p.numel() for p in test_load['network'].values() 
                                     if isinstance(p, torch.Tensor))
                    print(f"    → ✅ Formato válido ({num_params:,} parámetros)")
                else:
                    print(f"    → ⚠️  Falta key 'network'")
                    
            except Exception as e:
                print(f"    → ❌ Error al verificar: {type(e).__name__}: {str(e)[:50]}")
            print()
else:
    print("❌ No se encontraron checkpoints válidos\n")
    
    # Diagnosticar problema
    if os.path.exists(checkpoint_dir):
        all_items = os.listdir(checkpoint_dir)
        if all_items:
            print("📂 Contenido de output/model_dump/:")
            for item in all_items[:15]:
                item_path = os.path.join(checkpoint_dir, item)
                if os.path.isdir(item_path):
                    print(f"    📁 {item}/ (directorio - no válido)")
                else:
                    size_mb = os.path.getsize(item_path) / (1024 * 1024)
                    print(f"    📄 {item} ({size_mb:.1f} MB)")
            
            print("\n💡 Solución: Re-ejecuta la celda anterior de extracción")
        else:
            print("💡 Directorio vacío - verifica MODELS_DATASET_PATH")
    else:
        print("💡 Solución: Verifica que MODELS_DATASET_PATH es correcto")

print("\n" + "="*60)
print("💡 IMPORTANTE: Los checkpoints son archivos .pth modernos")
print("   Convertidos desde formato legacy a formato PyTorch estándar")
print("="*60)


🔍 Verificación de Checkpoints:

✅ Checkpoints encontrados:

  ✓ snapshot_70.pth (87.1 MB)
    → Usa CHECKPOINT_EPOCH = 70
    → Keys: ['epoch', 'network', 'optimizer']
    → ✅ Formato válido (7,596,986 parámetros)

  ✓ snapshot_83.pth (96.2 MB)
    → Usa CHECKPOINT_EPOCH = 83
    → Keys: ['epoch', 'network', 'optimizer']
    → ✅ Formato válido (8,391,354 parámetros)


💡 IMPORTANTE: Los checkpoints son archivos .pth modernos
   Convertidos desde formato legacy a formato PyTorch estándar


**⚠️ IMPORTANTE:** Ajusta el valor de `CHECKPOINT_EPOCH` en las siguientes celdas según el epoch de tu checkpoint extraído. En este caso detectamos `snapshot_83.pth`, así que usa `CHECKPOINT_EPOCH = 83`.

## 🚀 PASO 4: Ejecutar Testing

### Modelo L (Large)

In [22]:
# Diagnóstico: Verificar que todos los componentes están listos
import sys
import os

print("🔍 Verificación de Componentes:\n")

# 1. Dataset - Verificar estructura DENTRO del proyecto
project_root = '/kaggle/working/ConvNeXtPose'
data_dir = os.path.join(project_root, 'data')
h36m_path = os.path.join(data_dir, 'Human36M')

print(f"1. Dataset (estructura del proyecto):")
print(f"   Project root: {project_root}")
print(f"   data/ exists: {os.path.exists(data_dir)}")
print(f"   data/dataset.py: {os.path.exists(os.path.join(data_dir, 'dataset.py'))}")
print(f"   data/Human36M/ exists: {os.path.exists(h36m_path)}")

if os.path.exists(h36m_path):
    print(f"   - Human36M.py: {os.path.exists(os.path.join(h36m_path, 'Human36M.py'))}")
    print(f"   - annotations: {os.path.exists(os.path.join(h36m_path, 'annotations'))}")
    print(f"   - images/S9: {os.path.exists(os.path.join(h36m_path, 'images', 'S9'))}")
    print(f"   - images/S11: {os.path.exists(os.path.join(h36m_path, 'images', 'S11'))}")

# 2. Checkpoints
checkpoint_dir = os.path.join(project_root, 'output/model_dump')
print(f"\n2. Checkpoints: {checkpoint_dir}")
if os.path.exists(checkpoint_dir):
    checkpoints = [f for f in os.listdir(checkpoint_dir) if f.startswith('snapshot_')]
    if checkpoints:
        print(f"   Disponibles: {', '.join(checkpoints)}")
        # Extraer epoch del checkpoint
        import re
        for ckpt in checkpoints:
            match = re.search(r'snapshot_(\d+)', ckpt)
            if match:
                epoch = match.group(1)
                print(f"   💡 Usa CHECKPOINT_EPOCH = {epoch} en la siguiente celda")
    else:
        print("   ⚠️  No se encontraron checkpoints")
else:
    print("   ❌ Directorio no existe")

# 3. Estructura del proyecto
print(f"\n3. Estructura del proyecto:")
critical_files = ['main/config.py', 'common/base.py', 'data/dataset.py', 'data/Human36M/Human36M.py']
for file_path in critical_files:
    full_path = os.path.join(project_root, file_path)
    exists = os.path.exists(full_path)
    status = "✓" if exists else "❌"
    print(f"   {status} {file_path}")

print("\n" + "="*60)
if all(os.path.exists(os.path.join(project_root, f)) for f in critical_files):
    print("✅ Todos los checks pasaron - Listo para testing")
else:
    print("❌ Algunos archivos faltan - Revisa la configuración")
print("="*60)

🔍 Verificación de Componentes:

1. Dataset (estructura del proyecto):
   Project root: /kaggle/working/ConvNeXtPose
   data/ exists: True
   data/dataset.py: True
   data/Human36M/ exists: True
   - Human36M.py: True
   - annotations: True
   - images/S9: True
   - images/S11: True

2. Checkpoints: /kaggle/working/ConvNeXtPose/output/model_dump
   Disponibles: snapshot_70.pth, snapshot_83.pth
   💡 Usa CHECKPOINT_EPOCH = 70 en la siguiente celda
   💡 Usa CHECKPOINT_EPOCH = 83 en la siguiente celda

3. Estructura del proyecto:
   ✓ main/config.py
   ✓ common/base.py
   ✓ data/dataset.py
   ✓ data/Human36M/Human36M.py

✅ Todos los checks pasaron - Listo para testing


### Ejecutar Testing desde Python

**Estructura correcta:**
- ✅ Módulos Python originales en `data/` (dataset.py, Human36M.py)
- ✅ Dataset de Kaggle enlazado en `data/Human36M/` (images, annotations)
- ✅ `config.py` configura automáticamente los paths

### ⚠️ IMPORTANTE: Verificar GPU Antes de Testing

**El modelo REQUIERE GPU para correr en tiempo razonable.**

- ✅ **Con GPU T4 x2**: ~10-20 minutos
- ❌ **Con CPU**: ~10-20 HORAS (no recomendado)

**Cómo activar GPU en Kaggle:**
1. Panel derecho → **Settings**
2. **Accelerator** → Selecciona **GPU T4 x2** o **GPU P100**
3. Click **Save**
4. El notebook se reiniciará con GPU habilitada

In [23]:
# Verificar disponibilidad de GPU
import torch

print("🔍 Verificando hardware disponible...\n")
if torch.cuda.is_available():
    print(f"✅ GPU disponible: {torch.cuda.get_device_name(0)}")
    print(f"   Memoria: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print(f"\n💡 Tiempo estimado: 10-20 minutos")
    USE_GPU = True
else:
    print("❌ GPU NO disponible - usando CPU")
    print("\n⚠️  ADVERTENCIA: El testing en CPU puede tomar HORAS")
    print("   Se recomienda activar GPU T4 x2 en Kaggle")
    print("\n¿Continuar de todas formas? (no recomendado)")
    USE_GPU = False

🔍 Verificando hardware disponible...

✅ GPU disponible: Tesla T4
   Memoria: 15.8 GB

💡 Tiempo estimado: 10-20 minutos


In [25]:
# Testing con estructura correcta del proyecto
import sys
import os

os.chdir('/kaggle/working/ConvNeXtPose/main')
from config import cfg

# No llamar cfg.load_variant_config(...) por ahora
cfg.head_cfg = None
cfg.backbone_cfg = ([3,3,9,3],[48,96,192,384])
cfg.variant = 'XS'
cfg.depth_dim = 64

# cfg.load_variant_config(VARIANT)
cfg.set_args('0')
CHECKPOINT_EPOCH = 83  # ← AJUSTAR según tu checkpoint

print(f"\n{'='*60}")
print(f"  Testing ConvNeXtPose-{cfg.variant}")
print(f"{'='*60}\n")

# 4. AHORA importar los demás módulos (config ya configuró sys.path)
import torch
import torch.backends.cudnn as cudnn
from base import Tester
import numpy as np
from tqdm import tqdm

# Configurar CUDA solo si está disponible
if torch.cuda.is_available():
    cudnn.benchmark = True
    cudnn.deterministic = False
    cudnn.enabled = True
    print("✅ CUDA habilitado")
else:
    print("⚠️  Ejecutando en CPU (será muy lento)")
from collections import OrderedDict
import torch

def map_legacy_head_keys(state_dict):
    new_state = OrderedDict()
    for k, v in state_dict.items():
        if k.startswith('module.head.deconv_layers_'):
            layer = k.split('.')[2]           # deconv_layers_1 / 2 / 3
            suffix = k.split('.', 3)[-1]      # 0.weight, 1.bias, 2.weight…
            if suffix.startswith('0.'):
                new_k = k.replace('.0.', '.dwconv.')
            elif suffix.startswith('1.'):
                new_k = k.replace('.1.', '.norm.')
            elif suffix.startswith('2.'):
                new_k = k.replace('.2.', '.pwconv.')
            else:
                new_k = k
            new_state[new_k] = v
        else:
            new_state[k] = v
    return new_state
orig_make_model = Tester._make_model

def legacy_make_model(self, test_epoch):
    import os
    from torch.nn.parallel import DataParallel
    from model import get_pose_net
    from config import cfg

    self.test_epoch = test_epoch
    base = os.path.join(cfg.model_dir, f'snapshot_{test_epoch}')
    model_path = base + '.pth'
    if not os.path.exists(model_path):
        model_path = base + '.pth.tar'
    self.logger.info(f'Load checkpoint from {model_path}')

    model = get_pose_net(cfg, False, self.joint_num)
    model = DataParallel(model).cuda()

    ckpt = torch.load(model_path, map_location='cpu')
    state_dict = ckpt['network']
    state_dict = map_legacy_head_keys(state_dict)  # <-- renombrar claves

    model.load_state_dict(state_dict)
    self.logger.info('✓ Checkpoint cargado (modo legacy)')
    model.eval()
    self.model = model

# 5. Crear tester y ejecutar
Tester._make_model = legacy_make_model
tester = Tester()
tester._make_batch_generator()
tester._make_model(CHECKPOINT_EPOCH)
print(f"\n🚀 Ejecutando testing en epoch {CHECKPOINT_EPOCH}...\n")

preds = []
with torch.no_grad():
    for itr, input_img in enumerate(tqdm(tester.batch_generator)):
        coord_out = tester.model(input_img)
        
        if cfg.flip_test:
            from utils.pose_utils import flip
            flipped_input_img = flip(input_img, dims=3)
            flipped_coord_out = tester.model(flipped_input_img)
            flipped_coord_out[:, :, 0] = cfg.output_shape[1] - flipped_coord_out[:, :, 0] - 1
            for pair in tester.flip_pairs:
                flipped_coord_out[:, pair[0], :], flipped_coord_out[:, pair[1], :] = \
                    flipped_coord_out[:, pair[1], :].clone(), flipped_coord_out[:, pair[0], :].clone()
            coord_out = (coord_out + flipped_coord_out)/2.
        
        coord_out = coord_out.cpu().numpy()
        preds.append(coord_out)

# Evaluar
preds = np.concatenate(preds, axis=0)
print(f"\n📊 Evaluando {len(preds)} predicciones...\n")
tester._evaluate(preds, cfg.result_dir)

print(f"\n✅ Testing completado!")
print(f"📂 Resultados guardados en: {cfg.result_dir}")
print(f"\n💡 MPJPE esperado para XS: ~52.0 mm (Protocol 2)")
print(f"💡 PA-MPJPE esperado para XS: ~36.5 mm (Protocol 1)")


10-14 16:56:12 Creating dataset...


>>> Using GPU: 0

  Testing ConvNeXtPose-XS

✅ CUDA habilitado
Load data of H36M Protocol 2
creating index...
index created!
Get bounding box and root from groundtruth


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
10-14 16:56:28 Load checkpoint from /kaggle/working/ConvNeXtPose/main/../output/model_dump/snapshot_83.pth


📐 Arquitectura: ConvNeXtPose-XS
   Backbone: 384 canales de salida
   HeadNet: Legacy (2-UP + 1 sin upsampling)


<ipython-input-25-bb4940fbf4cf>:76: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(model_path, map_location='cpu')


RuntimeError: Error(s) in loading state_dict for DataParallel:
	size mismatch for module.head.deconv_layers_1.pwconv.weight: copying a param with shape torch.Size([512, 384, 1, 1]) from checkpoint, the shape in current model is torch.Size([256, 384, 1, 1]).
	size mismatch for module.head.deconv_layers_1.pwconv.bias: copying a param with shape torch.Size([512]) from checkpoint, the shape in current model is torch.Size([256]).
	size mismatch for module.head.deconv_layers_2.dwconv.weight: copying a param with shape torch.Size([512, 1, 3, 3]) from checkpoint, the shape in current model is torch.Size([256, 1, 3, 3]).
	size mismatch for module.head.deconv_layers_2.dwconv.bias: copying a param with shape torch.Size([512]) from checkpoint, the shape in current model is torch.Size([256]).
	size mismatch for module.head.deconv_layers_2.norm.weight: copying a param with shape torch.Size([512]) from checkpoint, the shape in current model is torch.Size([256]).
	size mismatch for module.head.deconv_

### Modelo M (Medium) - Opcional

In [ ]:
# Si tienes el checkpoint del modelo M, ejecuta esto:
# !python test.py --gpu 0 --epochs {CHECKPOINT_EPOCH} --variant M

## 📊 PASO 5: Verificar Resultados

In [ ]:
# Ver resultados generados
%cd ..
!ls -lh output/result/

# Leer log de resultados
import glob
log_files = glob.glob('output/log/*.log')
if log_files:
    latest_log = max(log_files, key=os.path.getctime)
    print(f"\n📄 Últimas líneas del log ({os.path.basename(latest_log)}):")
    !tail -n 20 {latest_log}
else:
    print("⚠️  No se encontraron logs")

## 📈 PASO 6: Análisis de Resultados

In [ ]:
import re

def extract_mpjpe_from_log(log_path):
    """Extrae el MPJPE del log"""
    with open(log_path, 'r') as f:
        content = f.read()
    
    # Buscar patrón de MPJPE
    pattern = r'MPJPE.*?(\d+\.\d+)'
    matches = re.findall(pattern, content)
    
    if matches:
        return float(matches[-1])  # Último valor
    return None

# Extraer resultados
log_files = glob.glob('output/log/*.log')
if log_files:
    latest_log = max(log_files, key=os.path.getctime)
    mpjpe = extract_mpjpe_from_log(latest_log)
    
    print("\n" + "="*60)
    print("  📊 RESULTADOS FINALES")
    print("="*60)
    
    if mpjpe:
        print(f"\n  MPJPE (Protocol 2): {mpjpe:.2f} mm")
        
        # Comparar con paper
        expected = {
            'L': 42.3,
            'M': 44.6
        }
        
        # Determinar variante
        for variant, expected_val in expected.items():
            diff = abs(mpjpe - expected_val)
            if diff < 5:
                print(f"\n  Variante detectada: {variant}")
                print(f"  Valor del paper: {expected_val:.2f} mm")
                print(f"  Diferencia: {mpjpe - expected_val:+.2f} mm")
                
                if diff < 2:
                    print("  ✅ Resultado excelente (dentro de ±2mm)")
                elif diff < 5:
                    print("  ✓ Resultado aceptable (dentro de ±5mm)")
                break
    else:
        print("\n⚠️  No se pudo extraer MPJPE del log")
        print("Revisa el log manualmente en output/log/")
    
    print("\n" + "="*60)
else:
    print("❌ No se encontraron logs de testing")

## 💾 PASO 7: Guardar Outputs

Kaggle guarda automáticamente todo en `/kaggle/working/`. Opcionalmente puedes copiar resultados específicos:

In [ ]:
# Crear resumen de resultados
import json
from datetime import datetime

summary = {
    'timestamp': datetime.now().isoformat(),
    'model': 'ConvNeXtPose-L',  # Cambiar según modelo testeado
    'checkpoint_epoch': CHECKPOINT_EPOCH,
    'dataset': 'Human3.6M Protocol 2',
    'mpjpe_mm': mpjpe if 'mpjpe' in locals() else None,
    'pytorch_version': torch.__version__,
    'cuda_version': torch.version.cuda if torch.cuda.is_available() else None
}

# Guardar resumen
with open('output/result/test_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

print("✓ Resumen guardado en output/result/test_summary.json")
print("\n📄 Contenido:")
print(json.dumps(summary, indent=2))

## 🎉 Testing Completado!

Los resultados están en:
- **Logs**: `output/log/`
- **Resultados**: `output/result/`
- **Resumen JSON**: `output/result/test_summary.json`

Todos los archivos en `/kaggle/working/ConvNeXtPose/output/` se guardarán automáticamente cuando hagas commit del notebook.